# SPR 2026 - Resubmit BERTimbau + Focal Loss v2

**Baseline BERTimbau + Focal Loss - Score: 0.79696** 🏆

Estratégias de melhoria:
- ✅ Focal Loss (gamma=2.0, alpha=0.25)
- ✅ 5 epochs com early stopping
- ✅ Warmup 10%

Este é o **modelo baseline** que superou TF-IDF.

---
## 📥 MODELO - Add Input → Models → `bertimbau-ptbr-complete` (fabianofilho)

**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. Settings → Accelerator → **GPU T4 x2**

---

In [ ]:
# ===== RESUBMIT BERTIMBAU + FOCAL LOSS v2 =====
# Baseline com score 0.79696

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# ======== CONFIGURAÇÕES ========
SEED = 42
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 5
LR = 2e-5
NUM_CLASSES = 7
FOCAL_GAMMA = 2.0
FOCAL_ALPHA = 0.25

DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

def find_model_path():
    base = '/kaggle/input'
    def has_config(path):
        return os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json'))
    def search_dir(directory, depth=0, max_depth=10):
        if depth > max_depth:
            return None
        try:
            for item in os.listdir(directory):
                path = os.path.join(directory, item)
                if os.path.isdir(path):
                    if has_config(path):
                        return path
                    result = search_dir(path, depth + 1, max_depth)
                    if result:
                        return result
        except:
            pass
        return None
    return search_dir(base)

MODEL_PATH = find_model_path()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}, Model: {MODEL_PATH}')
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
# Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        return focal_loss

criterion = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)

In [ ]:
# Dataset
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')

class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        enc = self.tokenizer(str(self.texts[idx]), truncation=True, max_length=self.max_len, padding='max_length', return_tensors='pt')
        item = {'input_ids': enc['input_ids'].squeeze(), 'attention_mask': enc['attention_mask'].squeeze()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].values, train_df['target'].values,
    test_size=0.1, random_state=SEED, stratify=train_df['target']
)

train_ds = TextDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_ds = TextDataset(val_texts, val_labels, tokenizer, MAX_LEN)
test_ds = TextDataset(test_df['report'].values, None, tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)
print(f'Train: {len(train_ds)}, Val: {len(val_ds)}, Test: {len(test_ds)}')

In [ ]:
# Modelo
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=NUM_CLASSES, local_files_only=True)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

In [ ]:
# Treinamento
def evaluate(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds.extend(outputs.logits.argmax(dim=1).cpu().numpy())
            if 'labels' in batch:
                labels.extend(batch['labels'].numpy())
    if labels:
        return f1_score(labels, preds, average='macro')
    return preds

best_f1 = 0
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}')
    
    for batch in pbar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    val_f1 = evaluate(model, val_loader)
    print(f'Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, Val F1={val_f1:.4f}')
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), '/kaggle/working/best_model.pt')
        print(f'  -> Melhor modelo! F1={best_f1:.4f}')

print(f'\nMelhor F1: {best_f1:.4f}')

In [ ]:
# Submissão
model.load_state_dict(torch.load('/kaggle/working/best_model.pt'))
predictions = evaluate(model, test_loader)

submission = pd.DataFrame({'ID': test_df['ID'], 'target': predictions})
submission.to_csv('/kaggle/working/submission.csv', index=False)
print('✅ submission.csv criado!')
print(submission['target'].value_counts().sort_index())